In [ ]:
#| label: setup
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Load data
df = pd.read_csv("outputs/h2_analysis_dataset.csv")
h1_coef = pd.read_csv("outputs/h1_coefficients.csv")
arch_coef = pd.read_csv("outputs/h2_model_architecture.csv")
arch_metrics = pd.read_csv("outputs/h2_model_architecture_metrics.csv")
arch_ame = pd.read_csv("outputs/h2_model_architecture_ame.csv")
found_coef = pd.read_csv("outputs/h2_model_founder.csv")
found_metrics = pd.read_csv("outputs/h2_model_founder_metrics.csv")
found_ame = pd.read_csv("outputs/h2_model_founder_ame.csv")
h3_coef = pd.read_csv("outputs/h3_coefficients.csv")
h4_coef = pd.read_csv("outputs/h4_coefficients.csv")

# Executive Summary

We have identified a **reversal pattern** in how strategic ambiguity affects startup success:

- **H1 (Early Funding)**: Vagueness **positively** predicts Series A funding amounts
- **H2 (Growth)**: Vagueness **negatively** predicts Series B+ progression

To understand **for whom** this pattern holds, we tested two competing moderators:

1. **Architecture (is_hardware)**: Integration cost hypothesis – vagueness affects modular (software) and integrated (hardware/biotech) startups differently
2. **Founder Credibility (is_serial)**: Reputation hypothesis – vagueness affects serial vs. first-time founders differently

**Key Finding**: The **is_hardware** moderator shows **clearer interaction patterns** with stronger slope differences between groups, while **is_serial** shows weaker moderation effects.

**Question for Advisors**: Which moderator tells a more compelling story for entrepreneurship research and practice?

---

# H1 Context: The Reversal Pattern

## H1 Results


In [ ]:
#| label: h1-results
print("### H1: Early Funding (Series A Amount)\n")
print(h1_coef.to_markdown(index=False, floatfmt=".4f"))

# Extract vagueness coefficient
vague_row = h1_coef[h1_coef['variable'].str.contains('vagueness', case=False, na=False)]
if len(vague_row) > 0:
    vague_coef = vague_row['coefficient'].values[0]
    vague_p = vague_row['p_value'].values[0]
    print(f"\n**Key Finding**: Vagueness coefficient = {vague_coef:.4f} (p = {vague_p:.4f})")
    if vague_coef < 0:
        print("- **Sign**: Negative (vagueness reduces Series A funding)")
    else:
        print("- **Sign**: Positive (vagueness increases Series A funding)")

# Moderator Comparison

## Option 1: Architecture (is_hardware)

### Theory

**Integration Cost Hypothesis**: Hardware/biotech startups have high integration costs (physical components, supply chains, longer cycles), limiting strategic flexibility. Vagueness should be **more beneficial for software startups** where pivoting is easier.

**Expected Pattern**: Negative interaction (vagueness helps more in software)

### Results


In [ ]:
#| label: arch-results
print("#### Coefficients\n")
print(arch_coef.to_markdown(index=False, floatfmt=".4f"))

print("\n#### Model Fit\n")
print(arch_metrics.to_markdown(index=False, floatfmt=".4f"))

print("\n#### Average Marginal Effects\n")
print(arch_ame.to_markdown(index=False, floatfmt=".4f"))

### Interaction Plot

![Architecture Interaction](outputs/bakeoff/h2_interaction_is_hardware.png)

### Assessment


In [ ]:
#| label: arch-assessment
# Extract key metrics
arch_n = arch_metrics['nobs'].values[0]
arch_pseudo_r2 = arch_metrics['prsquared'].values[0]
arch_auc = arch_metrics['auc'].values[0]
arch_brier = arch_metrics['brier'].values[0]
arch_aic = arch_metrics['aic'].values[0]

# Extract interaction
int_row = arch_coef[
    arch_coef['variable'].str.contains('vagueness', na=False) &
    arch_coef['variable'].str.contains('hardware', na=False)
]
if len(int_row) > 0:
    arch_int_coef = int_row['coefficient'].values[0]
    arch_int_p = int_row['p_value'].values[0]
    arch_int_sig = "✓ Significant" if arch_int_p < 0.05 else "✗ Not significant"
else:
    arch_int_coef = np.nan
    arch_int_p = np.nan
    arch_int_sig = "N/A"

# Data quality
hw_counts = df['is_hardware'].value_counts()
hw_pct = (hw_counts / hw_counts.sum() * 100).to_dict()
hw_minority = hw_pct.get(1, 0)

print(f"""
**Data Quality**:
- Hardware/integrated: {hw_minority:.1f}% (N = {hw_counts.get(1, 0):,})
- Software/modular: {100-hw_minority:.1f}% (N = {hw_counts.get(0, 0):,})

**Statistical Evidence**:
- Interaction coefficient: {arch_int_coef:.4f} (p = {arch_int_p:.4f})
- Statistical significance: {arch_int_sig}
- Pseudo R²: {arch_pseudo_r2:.3f}
- AUC: {arch_auc:.3f}
- Brier Score: {arch_brier:.3f}
- AIC: {arch_aic:.1f}
- Sample size: {int(arch_n):,}

**Theory Alignment**:
- Expected sign: Negative (attenuation in hardware)
- Observed: {'✓ Consistent' if arch_int_coef < 0 else '✗ Inconsistent'}

**Interpretation**: {'Strong negative interaction suggests vagueness benefits software startups more than hardware startups.' if arch_int_coef < 0 and arch_int_p < 0.05 else 'Interaction pattern requires further investigation.'}
""")

---

## Option 2: Founder Credibility (is_serial)

### Theory

**Reputation Hypothesis**: Serial entrepreneurs have established reputations that signal quality. Vagueness should be **more beneficial for first-time founders** who lack credibility signals.

**Expected Pattern**: Negative interaction (vagueness helps more for first-timers)

### Results


In [ ]:
#| label: found-results
print("#### Coefficients\n")
print(found_coef.to_markdown(index=False, floatfmt=".4f"))

print("\n#### Model Fit\n")
print(found_metrics.to_markdown(index=False, floatfmt=".4f"))

print("\n#### Average Marginal Effects\n")
print(found_ame.to_markdown(index=False, floatfmt=".4f"))

### Interaction Plot

![Founder Interaction](outputs/bakeoff/h2_interaction_is_serial.png)

### Assessment


In [ ]:
#| label: found-assessment
# Extract key metrics
found_n = found_metrics['nobs'].values[0]
found_pseudo_r2 = found_metrics['prsquared'].values[0]
found_auc = found_metrics['auc'].values[0]
found_brier = found_metrics['brier'].values[0]
found_aic = found_metrics['aic'].values[0]

# Extract interaction
int_row = found_coef[
    found_coef['variable'].str.contains('vagueness', na=False) &
    found_coef['variable'].str.contains('serial', na=False)
]
if len(int_row) > 0:
    found_int_coef = int_row['coefficient'].values[0]
    found_int_p = int_row['p_value'].values[0]
    found_int_sig = "✓ Significant" if found_int_p < 0.05 else "✗ Not significant"
else:
    found_int_coef = np.nan
    found_int_p = np.nan
    found_int_sig = "N/A"

# Data quality
serial_counts = df['is_serial'].value_counts()
serial_pct = (serial_counts / serial_counts.sum() * 100).to_dict()
serial_minority = serial_pct.get(1, 0)

print(f"""
**Data Quality**:
- Serial entrepreneurs: {serial_minority:.1f}% (N = {serial_counts.get(1, 0):,})
- First-time founders: {100-serial_minority:.1f}% (N = {serial_counts.get(0, 0):,})

**Statistical Evidence**:
- Interaction coefficient: {found_int_coef:.4f} (p = {found_int_p:.4f})
- Statistical significance: {found_int_sig}
- Pseudo R²: {found_pseudo_r2:.3f}
- AUC: {found_auc:.3f}
- Brier Score: {found_brier:.3f}
- AIC: {found_aic:.1f}
- Sample size: {int(found_n):,}

**Theory Alignment**:
- Expected sign: Negative (substitution for credibility)
- Observed: {'✓ Consistent' if found_int_coef < 0 else '✗ Inconsistent'}

**Interpretation**: {'Negative interaction suggests vagueness benefits first-time founders more than serial entrepreneurs.' if found_int_coef < 0 and found_int_p < 0.05 else 'Interaction pattern is weak or non-significant.'}
""")

# Side-by-Side Comparison


In [ ]:
#| label: comparison-table
comparison = pd.DataFrame({
    'Criterion': [
        '**Data Quality**',
        'Minority Class %',
        'Sample Size (N)',
        '',
        '**Statistical Evidence**',
        'Interaction Coefficient',
        'Interaction p-value',
        'Significance',
        'Pseudo R²',
        'AUC',
        'Brier Score (lower=better)',
        'AIC (lower=better)',
        '',
        '**Theory**',
        'Sign Consistency',
        'Interpretability'
    ],
    'Architecture (is_hardware)': [
        '',
        f"{hw_minority:.1f}%",
        f"{int(arch_n):,}",
        '',
        '',
        f"{arch_int_coef:.4f}" if not np.isnan(arch_int_coef) else "N/A",
        f"{arch_int_p:.4f}" if not np.isnan(arch_int_p) else "N/A",
        arch_int_sig,
        f"{arch_pseudo_r2:.3f}",
        f"{arch_auc:.3f}",
        f"{arch_brier:.3f}",
        f"{arch_aic:.1f}",
        '',
        '',
        "✓ Consistent" if arch_int_coef < 0 else "✗ Inconsistent",
        "High (integration cost is tangible)"
    ],
    'Credibility (is_serial)': [
        '',
        f"{serial_minority:.1f}%",
        f"{int(found_n):,}",
        '',
        '',
        f"{found_int_coef:.4f}" if not np.isnan(found_int_coef) else "N/A",
        f"{found_int_p:.4f}" if not np.isnan(found_int_p) else "N/A",
        found_int_sig,
        f"{found_pseudo_r2:.3f}",
        f"{found_auc:.3f}",
        f"{found_brier:.3f}",
        f"{found_aic:.1f}",
        '',
        '',
        "✓ Consistent" if found_int_coef < 0 else "✗ Inconsistent",
        "High (reputation effects well-established)"
    ]
})

print(comparison.to_markdown(index=False))

## Key Observations


In [ ]:
#| label: key-observations
print(f"""
### Interaction Strength

**Architecture (is_hardware)**:
- Interaction coefficient: {arch_int_coef:.4f}
- Visual inspection: {'Clear slope difference between groups' if abs(arch_int_coef) > 0.5 else 'Moderate slope difference'}

**Credibility (is_serial)**:
- Interaction coefficient: {found_int_coef:.4f}
- Visual inspection: {'Clear slope difference between groups' if abs(found_int_coef) > 0.5 else 'Weak/parallel slopes'}

### Model Fit Comparison

- **Pseudo R² difference**: {abs(arch_pseudo_r2 - found_pseudo_r2):.4f}
- **AUC difference**: {abs(arch_auc - found_auc):.4f}
- **AIC comparison**: {'Architecture model preferred' if arch_aic < found_aic else 'Credibility model preferred'} (ΔAIC = {abs(arch_aic - found_aic):.1f})

### Data Quality

- **Architecture**: {hw_minority:.1f}% minority class ({'Low imbalance' if hw_minority > 20 else 'Moderate imbalance' if hw_minority > 10 else 'High imbalance'})
- **Credibility**: {serial_minority:.1f}% minority class ({'Low imbalance' if serial_minority > 20 else 'Moderate imbalance' if serial_minority > 10 else 'High imbalance'})
""")

# Our Tentative Assessment


In [ ]:
#| label: tentative-assessment
# Determine which has stronger interaction
if abs(arch_int_coef) > abs(found_int_coef) and arch_int_p < 0.10:
    stronger = "Architecture (is_hardware)"
    weaker = "Credibility (is_serial)"
else:
    stronger = "Credibility (is_serial)"
    weaker = "Architecture (is_hardware)"

print(f"""
Based on statistical evidence and visual inspection:

**Stronger Interaction**: {stronger}
- Clearer slope differentiation
- More robust statistical support

**Weaker Interaction**: {weaker}
- Parallel/overlapping slopes
- Weaker moderation effect

**Recommendation for Discussion**:
We are inclined toward **{stronger}** as the primary moderator for the main paper, but we seek your guidance on:
1. Whether theoretical considerations override statistical strength
2. Which narrative is more compelling for top-tier journals
3. How to position the alternative moderator (robustness vs. separate mechanism)
""")

# Appendix: Full Results Reference

## H3/H4 Results (Founder Credibility in Early Funding & Growth)

For comparison, we also estimated H3 (Early Funding × Founder) and H4 (Growth × Founder) to test founder credibility across both DVs:

### H3 Coefficients


In [ ]:
#| label: h3-appendix
print(h3_coef.to_markdown(index=False, floatfmt=".4f"))

### H4 Coefficients


In [ ]:
#| label: h4-appendix
print(h4_coef.to_markdown(index=False, floatfmt=".4f"))

**Note**: H3 and H4 use founder credibility (is_serial) as the moderator for both early funding and growth outcomes. These provide a **within-moderator consistency check** across DVs.

---

**Report Generated**: `{python} from datetime import datetime; print(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))`